In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, \
                                    MaxPooling2D, BatchNormalization, Activation, \
                                    concatenate, Input, GlobalAveragePooling2D, \
                                    BatchNormalization, Flatten, Add, Input, InputLayer, \
                                    Activation, AveragePooling2D, Multiply

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, \
                                        EarlyStopping

from tensorflow.keras import backend

from tensorflow.keras.regularizers import l1, l2

In [2]:
# height, width, number of channel
def build_model(input_shape=(36, 36, 3, 2), drop_rate=0.3, hidden_units=128, output_size=1):
    input_layer = Input(input_shape)
    cur_input_layer, next_input_layer = \
                                [tf.squeeze(layer, axis=-1) for layer in tf.split(input_layer, num_or_size_splits=2, axis=-1)]
                                    
    # Appearance Model
    appearance_input_layer = cur_input_layer
    
    appr_layer1 = Conv2D(32, (3, 3), activation='tanh', padding='same')(appearance_input_layer)
    appr_layer2 = Conv2D(32, (3, 3), activation='tanh', padding='same')(appr_layer1)
    
    appr_norm1 = Conv2D(32, (1, 1), activation='sigmoid', padding='same')(appr_layer2)
    conv_sum1 = backend.sum(backend.sum(appr_norm1, axis=1), axis=1)
    appr_norm1 = appr_norm1.shape[1] * appr_norm1.shape[2] * appr_norm1 / (2 * conv_sum1)
    
    appr_layer3 = AveragePooling2D()(appr_layer2)
    appr_layer3 = Conv2D(32, (2, 2), activation='tanh', padding='same')(appr_layer3)
    appr_layer4 = Conv2D(32, (3, 3), activation='tanh', padding='same')(appr_layer3)
    appr_layer5 = Conv2D(64, (3, 3), activation='tanh', padding='same')(appr_layer4)
    
    appr_norm2 = Conv2D(64, (1, 1), activation='sigmoid', padding='same')(appr_layer5)
    conv_sum2 = backend.sum(backend.sum(appr_norm2, axis=1), axis=1)
    appr_norm2 = appr_norm2.shape[1] * appr_norm2.shape[2] * appr_norm2 / (2 * conv_sum2)
    
    # Motion Model
    # Layer 1
    motion_input_layer = (next_input_layer - cur_input_layer) / \
                            (next_input_layer + cur_input_layer)
    layer1 = Conv2D(32, (3, 3), activation='tanh', padding='same')(motion_input_layer)
    
    # Layer 2
    layer2 = Conv2D(32, (3, 3), activation='tanh', padding='same')(layer1)
    
    # L1 normalization
    motion_norm1 = Multiply()([appr_norm1, layer2])
    
    # Layer 3
    layer3 = AveragePooling2D()(motion_norm1)
    layer3 = Conv2D(32, (2, 2), activation='tanh', padding='same')(layer3)
    
    # Layer 4
    layer4 = Conv2D(32, (3, 3), activation='tanh', padding='same')(layer3)
    
    # Layer 5
    layer5 = Conv2D(64, (3, 3), activation='tanh', padding='same')(layer4)
    
    # L1 normalization
    motion_norm2 = Multiply()([appr_norm2, layer5])
    
    # Layer 6
    layer6 = AveragePooling2D()(motion_norm2)
    layer6 = Conv2D(64, (2, 2), activation='tanh', padding='same')(layer6)
    
    # Layer 7
    layer7 = Flatten()(layer6)
    
    # Layer 8
    layer8 = Dense(hidden_units, activation='relu')(layer7)    
    
    # Layer 9
    output_layer = Dense(output_size, activation='relu')(layer8)
    
    return Model(input_layer, output_layer)

In [3]:
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 36, 36, 3, 2 0                                            
__________________________________________________________________________________________________
tf_op_layer_split (TensorFlowOp [(None, 36, 36, 3, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Squeeze (TensorFlow [(None, 36, 36, 3)]  0           tf_op_layer_split[0][0]          
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 36, 36, 32)   896         tf_op_layer_Squeeze[0][0]        
______________________________________________________________________________________________